In [41]:
import pyspark
import string
from pecab import PeCab
import pandas as pd
import numpy as np
from pyspark.sql.functions import pandas_udf, monotonically_increasing_id, regexp_replace, sum, asc
from pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType
import re

In [3]:
# local[*] : 모든 코어를 사용하겠다, local[4] : 4개의 코어를 사용하겠다.

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("WordCount")\
        .getOrCreate()


In [4]:
%%time

# Define the schema of the DataFrame
# schema = StructType([
#                      StructField("date", StringType(), True), \
#                      StructField("idol", StringType(), True), \
#                      StructField("press", StringType(), True), \
#                      StructField("title", StringType(), True), \
#                      StructField("summary", IntegerType(), True), \
#                      StructField("content", IntegerType(), True), \
#                      StructField("link", IntegerType(), True), \
#                      StructField("thumbnail_link", IntegerType(), True)])
data_name= "test_csv"
# Load the text data into a DataFrame
df = spark.read.options(header='True', inferSchema='True', delimiter='|',quote='"', multiLine="true").option("encoding", "UTF-8").csv(f"file:///home/j8a507/cluster/news/{data_name}.csv")
# df.collect()

CPU times: user 51.6 ms, sys: 9.96 ms, total: 61.5 ms
Wall time: 1min 28s


In [5]:
@pandas_udf(StringType())
def rm_space_in_quote_udf(x: pd.Series) -> pd.Series:
    def rm_space_in_quote(string):
        encoded_string = string.encode('utf-8')
        decoded_string = encoded_string.decode('utf-8')
        no_space_string = re.sub(r"'[^']*'", lambda m: m.group().replace(" ", ""), decoded_string)
        return no_space_string

    return x.apply(rm_space_in_quote)

In [6]:
%%time
df = df.withColumn("content", rm_space_in_quote_udf(df["content"]))
df.collect()

[Row(date='2023-03-03', idol='TWICE', press='연합뉴스TV', title="트와이스, K팝 여가수 최초 '빌보드 위민 인 뮤직' 수상", summary=' "걸그룹 트와이스가 K팝 여가수 최초로 \'빌보드 위민 인 뮤직\'에서 수상했습니다.소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 \'빌보드 위민 인 뮤직\'"', content="걸그룹 트와이스가 K팝 여가수 최초로 '빌보드위민인뮤직'에서 수상했습니다. 소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 '빌보드위민인뮤직' 시상식에 참석해 '브레이크스루아티스트' 부문에서 수상했다고 밝혔습니다. '브레이크스루아티스트' 부문은 음악 시장에서 의미있는 도전을 해내고 두각을 드러낸 가수에게 주는 상입니다.빌보드가 주최하는 이 시상식은 한 해 음악 산업에 큰 영향을 끼친 여성 아티스트와 크리에이터, 프로듀서 등에 시상합니다.신새롬 기자 (romi@yna.co.kr)연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23", link='https://entertain.naver.com/now/read?oid=422&aid=0000586954', thumbnail_link='https://mimgnews.pstatic.net/image/origin/422/2023/03/03/586954.jpg?type=nf124_82_q90'),
 Row(date='2023-03-03', idol='NewJeans', press='스포츠조선', title="뉴진스, 데뷔 전 모습 공개에 '얼마나 더 어렸던 거야?', 민지·혜인, 의욕 넘치고 생기가 가득", summary=' ""뉴진스에게 이리 \'애기애기\'하던 시절이 있었다니……이 영상 박제해요! 민희진 대표가 자신의 개인 계정에 뉴진스의 민지 혜인의 데뷔 전 영상을 올려 눈길을 끈다. 영상 속 민지와 혜인은 지금보다 훨씬 앳되보

In [7]:
user_dict= {"뉴진스","트와이스"}

# 사용자 정의사전 만들기
@pandas_udf(ArrayType(StringType()))
def make_user_dict_udf(x: pd.Series) -> pd.Series:
    def make_user_dict(string):
        encoded_string = string.encode('utf-8')
        decoded_string = encoded_string.decode('utf-8')
        substrings = re.findall(r"'(.*?)'", decoded_string)
        return substrings
    return x.apply(make_user_dict)

df_dict = df.withColumn("user_dict_row", make_user_dict_udf(df["content"]))
# df_dict.collect()
for w in df_dict.toLocalIterator():
    user_dict.update(w["user_dict_row"])
print(user_dict)


# non_empty_lists = user_words.filter(lambda x: len(x) > 0)
# flattened_lists = non_empty_lists.flatMap(lambda x: x)
# user_dict += flattened_lists.map(lambda x: x.replace(" ","")).collect()

{'디토', '빌보드우먼인뮤직', '브레이크스루아티스트', '뉴진스', '트와이스', '두분한번일어나주시죠', '빌보드위민인뮤직', '글로벌영향력있는여성', '애기애기', '2023빌보드우먼인뮤직'}


In [9]:
# user_dict : 사용자 정의 단어
pecab = PeCab(user_dict=user_dict)

In [10]:
@pandas_udf(StringType())
def clean_str_udf(x: pd.Series) -> pd.Series:
    def clean_str(x):
        punc = string.punctuation
        for ch in punc:
            x = x.replace(ch, '')
        return x
    return x.apply(clean_str)

In [24]:
%%time
df = df.withColumn("content", clean_str_udf(df["content"]))
df.collect()

CPU times: user 44.3 ms, sys: 0 ns, total: 44.3 ms
Wall time: 15.4 s


[Row(date='2023-03-03', idol='TWICE', press='연합뉴스TV', title="트와이스, K팝 여가수 최초 '빌보드 위민 인 뮤직' 수상", summary=' "걸그룹 트와이스가 K팝 여가수 최초로 \'빌보드 위민 인 뮤직\'에서 수상했습니다.소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 \'빌보드 위민 인 뮤직\'"', content='걸그룹 트와이스가 K팝 여가수 최초로 빌보드위민인뮤직에서 수상했습니다 소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 빌보드위민인뮤직 시상식에 참석해 브레이크스루아티스트 부문에서 수상했다고 밝혔습니다 브레이크스루아티스트 부문은 음악 시장에서 의미있는 도전을 해내고 두각을 드러낸 가수에게 주는 상입니다빌보드가 주최하는 이 시상식은 한 해 음악 산업에 큰 영향을 끼친 여성 아티스트와 크리에이터 프로듀서 등에 시상합니다신새롬 기자 romiynacokr연합뉴스TV 기사문의 및 제보  카톡라인 jebo23', link='https://entertain.naver.com/now/read?oid=422&aid=0000586954', thumbnail_link='https://mimgnews.pstatic.net/image/origin/422/2023/03/03/586954.jpg?type=nf124_82_q90'),
 Row(date='2023-03-03', idol='NewJeans', press='스포츠조선', title="뉴진스, 데뷔 전 모습 공개에 '얼마나 더 어렸던 거야?', 민지·혜인, 의욕 넘치고 생기가 가득", summary=' ""뉴진스에게 이리 \'애기애기\'하던 시절이 있었다니……이 영상 박제해요! 민희진 대표가 자신의 개인 계정에 뉴진스의 민지 혜인의 데뷔 전 영상을 올려 눈길을 끈다. 영상 속 민지와 혜인은 지금보다 훨씬 앳되보이는 모습.이미지 원본보기사진 출처=

['걸', '그룹', '트와이스', '팝', '가수', '최초', '빌보드위민인뮤직', '수상', '소속', '사', '엔터', '테', '인먼', '걸', '그룹', '트와이스', '현지', '시간', '일', '미국', '로스앤젤레스', '유튜브', '시어터', '빌보드위민인뮤직', '시상식', '참석', '브레이크스루아티스트', '부문', '수상', '브레이크스루아티스트', '부문', '음악', '시장', '의미', '도전', '두각', '가수', '상', '빌보드', '주최', '시상식', '해', '음악', '산업', '영향', '여성', '아티스트', '크리에이터', '프로듀서', '등', '시상', '새롬', '기자', '연합뉴스', '기사문', '제보', '카톡', '라인']


In [38]:
from sklearn.feature_extraction.text import CountVectorizer

def word_count(nouns):
    vect = CountVectorizer()
    document_term_matrix = vect.fit_transform(nouns)

    tf = pd.DataFrame(document_term_matrix.toarray(), columns=vect.get_feature_names_out())

    D = len(tf)
    df = tf.astype(bool).sum(axis=0)
    idf = np.log((D+1) / (df+1)) + 1             # IDF (Inverse Document Frequency)

    # TF-IDF (Term Frequency-Inverse Document Frequency)
    tfidf = tf * idf                      
    tfidf = tfidf / np.linalg.norm(tfidf, axis=1, keepdims=True)
    return tfidf

In [45]:
words = pecab.nouns("걸그룹 트와이스가 K팝 여가수 최초로 빌보드위민인뮤직에서 수상했습니다 소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 빌보드위민인뮤직 시상식에 참석해 브레이크스루아티스트 부문에서 수상했다고 밝혔습니다 브레이크스루아티스트 부문은 음악 시장에서 의미있는 도전을 해내고 두각을 드러낸 가수에게 주는 상입니다빌보드가 주최하는 이 시상식은 한 해 음악 산업에 큰 영향을 끼친 여성 아티스트와 크리에이터 프로듀서 등에 시상합니다신새롬 기자 romiynacokr연합뉴스TV 기사문의 및 제보  카톡라인 jebo23")
pdf = spark.createDataFrame(word_count(words).dropna())
# pdf.collect()
sums = pdf.agg(*[sum(c).alias(c) for c in pdf.columns])
sums.collect()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


[Row(가수=2.0, 그룹=2.0, 기사문=1.0, 기자=1.0, 도전=1.0, 두각=1.0, 라인=1.0, 로스앤젤레스=1.0, 미국=1.0, 부문=2.0, 브레이크스루아티스트=2.0, 빌보드=1.0, 빌보드위민인뮤직=2.0, 산업=1.0, 새롬=1.0, 소속=1.0, 수상=2.0, 시간=1.0, 시상=1.0, 시상식=2.0, 시어터=1.0, 시장=1.0, 아티스트=1.0, 엔터=1.0, 여성=1.0, 연합뉴스=1.0, 영향=1.0, 유튜브=1.0, 음악=2.0, 의미=1.0, 인먼=1.0, 제보=1.0, 주최=1.0, 참석=1.0, 최초=1.0, 카톡=1.0, 크리에이터=1.0, 트와이스=2.0, 프로듀서=1.0, 현지=1.0)]

In [19]:
@pandas_udf(ArrayType(MapType()))
def pecab_nouns_udf(x: pd.Series) -> pd.Series:
    def pecab_nouns(x):
        nouns = pecab.nouns(x)
        
        return nouns
    return x.apply(pecab_nouns)

df_words = df.withColumn("words", pecab_nouns_udf(df["content"]))
df_words.collect()

[Row(date='2023-03-03', idol='TWICE', press='연합뉴스TV', title="트와이스, K팝 여가수 최초 '빌보드 위민 인 뮤직' 수상", summary=' "걸그룹 트와이스가 K팝 여가수 최초로 \'빌보드 위민 인 뮤직\'에서 수상했습니다.소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 \'빌보드 위민 인 뮤직\'"', content='걸그룹 트와이스가 K팝 여가수 최초로 빌보드위민인뮤직에서 수상했습니다 소속사 JYP엔터테인먼트는 걸그룹 트와이스가 현지시간으로 지난 1일 미국 로스앤젤레스 유튜브 시어터에서 열린 빌보드위민인뮤직 시상식에 참석해 브레이크스루아티스트 부문에서 수상했다고 밝혔습니다 브레이크스루아티스트 부문은 음악 시장에서 의미있는 도전을 해내고 두각을 드러낸 가수에게 주는 상입니다빌보드가 주최하는 이 시상식은 한 해 음악 산업에 큰 영향을 끼친 여성 아티스트와 크리에이터 프로듀서 등에 시상합니다신새롬 기자 romiynacokr연합뉴스TV 기사문의 및 제보  카톡라인 jebo23', link='https://entertain.naver.com/now/read?oid=422&aid=0000586954', thumbnail_link='https://mimgnews.pstatic.net/image/origin/422/2023/03/03/586954.jpg?type=nf124_82_q90', words=['걸', '그룹', '트와이스', '팝', '가수', '최초', '빌보드위민인뮤직', '수상', '소속', '사', '엔터', '테', '인먼', '걸', '그룹', '트와이스', '현지', '시간', '일', '미국', '로스앤젤레스', '유튜브', '시어터', '빌보드위민인뮤직', '시상식', '참석', '브레이크스루아티스트', '부문', '수상', '브레이크스루아티스트', '부문', '음악', '시장', '의미', '도전', '두각', '가수', '상', '빌

In [433]:
# 단어로 나누기
# Return a new RDD by first applying a function to all elements of this RDD,
# and then flattening the results
article_rdd = article_rdd.flatMap(lambda article: pecab.nouns(article))



In [434]:
# 공백문자 없애기
article_rdd = article_rdd.filter(lambda x: x!='')

In [435]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

def word_count(nouns):
    vect = CountVectorizer()
    document_term_matrix = vect.fit_transform(nouns)

    tf = pd.DataFrame(document_term_matrix.toarray(), columns=vect.get_feature_names_out())

    D = len(tf)
    df = tf.astype(bool).sum(axis=0)
    idf = np.log((D+1) / (df+1)) + 1             # IDF (Inverse Document Frequency)

    # TF-IDF (Term Frequency-Inverse Document Frequency)
    tfidf = tf * idf                      
    tfidf = tfidf / np.linalg.norm(tfidf, axis=1, keepdims=True)
    tfidf.to_csv('output.csv', encoding = 'utf-8-sig')

In [ ]:
# # 공백문자 없애기
# article_rdd = article_rdd.filter(lambda x: x!='')

# # 나왔던 단어 중복제거하며 저장
# article_count = article_rdd.map(lambda word:(word,1))

# # Reduce By Key
# # Key에 대해 count
# article_count_RBK = article_count.reduceByKey(lambda x,y: (x+y)).sortByKey()

# # (key,var) -> (var,key) 로 변경
# article_count_RBK = article_count_RBK.map(lambda x:(x[1],x[0]))

# #
# article_count_RBK.sortByKey(False).take(10)